In [1]:
import numpy as np
import multiprocessing as mp
import pickle 
import pandas as pd
mp.set_start_method('spawn',True);
import torch
torch.multiprocessing.set_start_method('spawn',True);
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from itertools import product
from tqdm import tqdm
import time

class Machine_Replacement:
    def __init__(self,rep_cost=0.7,nS=6,nA=2):
        self.nS = nS;
        self.nA = nA;
        self.cost = np.linspace(0.1, 0.99,nS);
        self.rep_cost = rep_cost;
    def gen_probability(self):
        self.P = np.zeros((self.nA,self.nS,self.nS));
        for i in range(self.nS):
            for j in range(self.nS):
                if(i<=j):
                    self.P[0,i,j]=(i+1)*(j+1);
                else:
                    continue;
            self.P[0,i,:]=self.P[0,i,:]/np.sum(self.P[0,i,:])
            self.P[1,i,0]=1;
        return self.P;
    def gen_reward(self):
        self.R=np.zeros((self.nA,self.nS,self.nS));
        for i in range(self.nS):
            self.R[0,i,:] = self.cost[i];
            self.R[1,i,0] = self.rep_cost+self.cost[0];
        return self.R;
    def gen_expected_reward(self):
        self.R = np.zeros((self.nA,self.nS));
        for i in range(self.nS):
            self.R[0,i] = self.cost[i];
            self.R[1,i] = self.rep_cost + self.cost[0];
        return self.R;

In [15]:
class get_hyperparameters:
    def __init__(self):
        self.T = 250000;
        self.runs = 3;
        self.lr = 0.1;
        self.batch_size = 50;
        self.start = 0;
        self.nS = 10;
        self.nA = 2;
        self.rep_cost = 0.7
        self.alpha = 0.5
        self.gamma = 0.5
        self.beta = 1;
    
    def ret_hyperparameters(self):
        return (self.T,self.runs,self.lr,self.batch_size,self.start,self.nS,self.nA,self.rep_cost,self.alpha,self.gamma,self.beta)

In [16]:
class weights(nn.Module):
    def __init__(self,input_size,output_size,hidden_size = 0):
        super(weights,self).__init__()
        self.input_size = input_size;
        self.hidden_size = hidden_size;
        self.output_size = output_size;
        if(hidden_size!=0):
            self.linear1 = nn.Linear(self.input_size, self.hidden_size, bias=False)
            self.linear2 = nn.Linear(self.hidden_size, self.output_size, bias=False)
        else:
            self.linear1 = nn.Linear(self.input_size, self.output_size, bias=False)
    '''
        forward(): We accept a state 's' as input. Then we convert this into one hot encoding which is accomplished by first two lines.
        Further we convert this one_hot vector 's' into pytorch tensor and then pass it through the network to obtain a output which is returned 
    '''
    def forward(self,state):
        s = np.zeros(self.input_size);
        #print(state,end='===>');
        s[state] = 1;
        state = torch.FloatTensor(s).to(device)
        #print(state);
        if(self.hidden_size == 0):
            output = torch.exp(self.linear1(state)) #To ensure that the outputs are always positive. giving Relu will cause problems.
        else:
            output = torch.exp(self.linear2(torch.exp(self.linear1(state))));
        return output
    
    def forward_batch(self,states):
        s = np.zeros((len(states),self.input_size));
        #print(state,end='===>');
        for i in range(len(states)):
            s[i][states[i]] = 1
        states = torch.FloatTensor(s).to(device)
        #print(state);
        if(self.hidden_size == 0):
            output = torch.exp(self.linear1(states)) #To ensure that the outputs are always positive. giving Relu will cause problems.
        else:
            output = torch.exp(self.linear2(torch.exp(self.linear1(state))));
        return output
    
    def fast_forward(self,s1,ns1,s2,ns2):
        return self.forward(s1),self.forward(ns1),self.forward(s2),self.forward(ns2);

In [17]:
class Target_Policy:
    '''
        First we create an initiualizer function namely a constructor to initialize the variables
        with initial data values
    '''
    def __init__(self,S,A,P,R,start):
        self.S=S # represant the states of the MDP
        self.nS = len(S) # Reperesants the number of states of the MDP
        self.nA = len(A);# Represants the number of actions in the MDP
        self.P=P # Represants the true Probability function
        self.R=R # Represants the true Reward  function
        self.A=A;# Represnats the Action Space
        self.K_pol = []; # To store all the policies
        self.s_start=start # Store the start state 
    '''
        In the generate_next_state(), we are generating our next state to be visited based on the following input parameters
        s : Current state
        a : Current action
    '''    
    def generate_next_state(self,s,a):
        #p = np.zeros(self.nS)
        p = self.P[a][s][:] # extrcat all the probabilities of the different statestransition given current s and a
        #print(p);
        return (np.argmax(np.random.multinomial(1,p)))
    
    '''
        Single function to find the plot between the cumulative regret generated by different algorithms
        Parameters:
            reg_list : A list containing the regret value at different runs instances averaged over several time
    '''    
    def plot_data(self,reg_list):
        plt.plot(np.arange(len(reg_list)),np.cumsum(np.array(reg_list)),marker = '+'); 
    '''
        Function to find the optimum policy out of the K policies found out.
        Parameters:
            runs : To find for how many runs the current policy to be runned
            T : Each run consisiting of how many time steps to find the average reward for each policy in one run
            Time complexity : O(#(policies) x #(episode runs) x #(number of Time steps in one episode))
    '''
    def find_optimum_policy(self):
        self.find_policies(); #Call the find_policies() to find all the policies and store it in 'self.K' list
        final_R = np.zeros(len(self.K_pol));
        for idx,pol in enumerate(self.K_pol):
            #policy = self.one_hot(pol);
            beh_obj = beh_pol_sd(self.P, pol, self.nS, self.nA)
            state_distribution = beh_obj.state_distribution_simulated(1);
            final_R[idx] = sum([state_distribution[state] *self.R[int(pol[state]),state] for state in range(self.nS)]);
        for l_pol in range(len(self.K_pol)):
            print(self.K_pol[l_pol],"    ====>    ",final_R[l_pol]); # Display the the expected reward for each policy
        return (final_R,self.K_pol[np.argmin(final_R)],np.min(final_R));# Return the minimum reward, the policy number which gives the minimum reward and the policy that gives minimum reward
    
    def find_policies(self):
        self.K_pol = [];
        pol=np.zeros(self.nS) # First policy is all 0's
        self.K_pol.append(np.array(pol)); # append it to our K_policy list namely self.K
        for i in reversed(range(self.nS)):
            pol[i] = 1; # Come from the end and since the structure is thresholding nature so make each position 1 from 0 and append the same
            print(pol);
            self.K_pol.append(np.array(pol));
        print(len(self.K_pol)," policies found");

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class average_case_distribution:
    def __init__(self,nS,nA,behaviour_policy,state,lr,batch_size):
        self.nS = nS
        self.nA = nA
        self.behaviour_policy = behaviour_policy;
        self.state = state;
        self.lr = lr
        self.W_loss = 0
        self.weight_obj = weights(nS,1).to(device);
        self.W_loss = 0;
        self.batch_size = batch_size
    def set_target_policy(self,target_pol):
        self.target_policy = target_pol;
        self.optimizerW = optim.Adam(self.weight_obj.parameters(),lr = self.lr);
        self.batch=[];
    def show_policy(self):
        print(self.target_policy);
    def set_batch(self,data):
        self.data = data;
        self.T = len(data);
    def get_batch(self):
        if(self.T<=self.batch_size):
            return self.data
        else:
            i = 1;
            j = np.random.randint(0,self.T,self.batch_size);
            batch = [[self.data[k][0],self.data[k][1],self.data[k][2]] for k in j]
            '''while(i<=self.batch_size):
                if(np.random.random()<=0.5):
                    batch.append([self.data[j][0],self.data[j][1],self.data[j][2]])
                    j = (j+1)%self.T;
                i+=1;'''
            return batch; 
    
    def get_w(self,data,weight_obj,m,pair=0):
        eps = 0.00000000001
        if(pair == 1):
            Z_w_state = 0;
            for i in range(len(data)):
                val = weight_obj(data[i][0]);
                #print(val);
                Z_w_state+=val;
            #print(Z_w_state.detach().numpy()[0]/self.batch_size);
            return (Z_w_state.detach().numpy()[0]/self.batch_size)+eps;
        else:
            state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2 = list(),list(),list(),list(),list(),list(),list(),list();
            K = list();
            for i in range(len(data)):
                sample1 = data[i][0];
                sample2 = data[i][1];
                state1.append(sample1[0]);
                #print(sample1);
                w_state1.append(weight_obj(sample1[0]));
                w_next_state1.append(weight_obj(sample1[2]));
                state2.append(sample2[0]);
                w_state2.append(weight_obj(sample2[0]));
                w_next_state2.append(weight_obj(sample2[2]));
                beta1.append(self.target_policy[sample1[0],sample1[1]]/self.behaviour_policy[sample1[0],sample1[1]]);
                beta2.append(self.target_policy[sample2[0],sample2[1]]/self.behaviour_policy[sample2[0],sample2[1]]);
                K.append(sample1[2]==sample2[2]);
            return (state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2,K);
    def get_w_updated(self,data,weight_obj,m,Z_w_state,pair=0): #data should be batch. note all inputs to be passed 
        if(pair == 1):
            Z_w_state = 0;
            for i in range(len(data)):
                val = weight_obj(data[i][0]);
                #print(val);
                Z_w_state+=val;
            #print(Z_w_state.detach().numpy()[0]/self.batch_size);
            Z_w_state = Z_w_state.cpu().detach().numpy()[0]/self.batch_size;
            if(Z_w_state<0.00000000000005):
                Z_w_state+=0.000000000001;
            return Z_w_state;
        else:
            data = torch.tensor(data)
#             print(data[:,0])
#             print(data[:,2])
#             print(data[:,0].size())
            new_data_current = torch.reshape(data[:,0],(50,1))
#             w_current_state = weight_obj.forward_batch(new_data_current)
            
            new_data_next = torch.reshape(data[:,2],(50,1))
            new_data = torch.cat((new_data_current,new_data_next),0)
            w = weight_obj.forward_batch(new_data)
            w = w/Z_w_state
#             print(w)
#             print(new_data.size())
#             print(w.size())
            w = torch.reshape(w,(2,50))
            beta = torch.zeros(50,1)
            for i in range(len(data)):
                beta[i] = self.target_policy[data[i][0],data[i][1]]/self.behaviour_policy[data[i][0],data[i][1]]
            
            K = torch.zeros(50,50)
            for i in range(len(data)):
                for j in range(len(data)):
                    K[i][j] = (data[i][2] == data[j][2])
#             print("here")
#             print(w.size())
#             print(beta.size())
#             print(K.size())
            w_0 = torch.reshape(w[0], (50,1))
            w_1 = torch.reshape(w[1], (50,1))
#             print(w_0)
#             print(beta)
#             print(w_1)
#             print(w_0 * beta - w_1)
#check here only
            delta = w_0 * beta - w_1
            temp1 = torch.matmul(K,delta) #50,50:50,1 = 50, 1
            final_sum = torch.matmul(torch.transpose(delta,0,1),temp1)
#             print(w_0)
#             print(w_1)
#             print(Z_w_state)
#             print(final_sum)
#             print(".........")
            return final_sum
    def update_state_distribution_ratio(self):
        self.batch = self.get_batch();
        batch=self.batch;
        #eps = 0.0004;
        for _ in range(100):
            start = time.time()
            Z_w_state = self.get_w(batch, self.weight_obj, len(batch),1);
#             state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2,K = self.get_w(pairs, self.weight_obj, len(batch));
            self.w_loss = 0
            self.w_loss = self.get_w_updated(batch,self.weight_obj,len(batch),Z_w_state)
#             print(time.time() - start)
#             print("*******")
#             for i in range(len(state1)):
#                 self.w_loss+=(beta1[i]*(w_state1[i]/Z_w_state) - (w_next_state1[i]/Z_w_state))*(beta2[i]*(w_state2[i]/Z_w_state)-(w_next_state2[i]/Z_w_state))*K[i];
#             print("sourav")
#             print(self.w_loss)
            self.w_loss/=(2*self.batch_size);
            start = time.time()
            self.optimizerW.zero_grad();
            self.w_loss.backward(); #Improving the forward pass computation
            self.optimizerW.step();
            self.optimizerW.zero_grad();
#             print(time.time() - start)
#            self.Z.append(Z_w_state)
            
#        self.loss.append(self.w_loss.cpu().detach().numpy()[0]);
#        state_dist=[];

In [19]:
class beh_pol_sd:
    def __init__(self,P,policy,nS,nA):
        self.P = P
        self.policy = policy
        self.nS = nS;
        self.nA = nA;
    
    def onehot(self):
        pol = np.zeros((self.nS,self.nA));
        for i in range(self.nS):
            pol[i][int(self.policy[i])]=1;
        return pol;
    def find_transition_matrix(self,onehot_encode=1):
        if(onehot_encode==1):
            self.policy = self.onehot()
        T_s_s_next = np.zeros((self.nS,self.nS));
        for s in range(self.nS):
            for s_next in range(self.nS):
                for a in range(self.nA):
                    #print(s,s_next,a);
                    #print(T[a,s,s_next]);
                    T_s_s_next[s,s_next]+=self.P[a,s,s_next]*self.policy[s,a];
        return T_s_s_next;
    def state_distribution_simulated(self,onehot_encode=1):
        P_policy = self.find_transition_matrix(onehot_encode)
        #print(P_policy);
        P_dash = np.append(P_policy - np.eye(self.nS),np.ones((self.nS,1)),axis=1);
        #print(P_dash);
        P_last = np.linalg.pinv(np.transpose(P_dash))[:,-1]
        return P_last;


In [20]:
def one_hot(target_policy,nS,nA):
  nPOL = len(target_policy);
  one_hot_target_policy = []
  for i in range(nPOL):
    pol=np.zeros((nS,nA));
    for j in range(nS):
      pol[j] = 1;
    one_hot_target_policy.append(pol);
  return np.array(one_hot_target_policy);

In [21]:
def simulate_episode(T,state,behaviour_policy,P,batch_size,run):
  #global P,behaviour_policy,batch_size;
  data={};temp=[];
  for t in range(1,T+1): 
    action = np.argmax(np.random.multinomial(1,behaviour_policy[state,:]))
    next_state = np.argmax(np.random.multinomial(1,P[action,state,:]));
    state = next_state;
    temp.append([state,action,next_state]);
    if(t%batch_size==0):
      #print(t);
      data[int(t/batch_size)-1] = temp[:];
  #print(len(data));
  with open('Data_used_'+str(run),'wb') as f:
    pickle.dump(temp,f);
  f.close();
  return data;

In [22]:
def softmax(theta):
  theta = np.exp(theta);
  sum = np.sum(theta)
  return theta/sum;

In [23]:
def get_rho(obj,nPOL,nS,nA,R,behaviour_policy_state_distribution,target_policy,sampled_policy):
    rho = np.zeros(nPOL)
    pol=sampled_policy
    state_dist = [];
    for i in range(nS):
        w_state = obj[pol].weight_obj(i);
        w_state = w_state.cpu().detach().numpy()[0];
        state_dist.append(w_state);
    sd = np.array(state_dist);
    sd = sd * behaviour_policy_state_distribution;
    sd = sd/np.sum(sd);
    rho[pol] = sum([sd[s] *R[target_policy[pol,s],s] for s in range(nS)]);
    return rho;

In [24]:
def processing(P,R,nPOL,nS,nA,T,behaviour_policy,behaviour_policy_state_distribution,target_policy,run,lr,beta,batch_size,alpha):
    start = 0;
    one_hot_target_policy = one_hot(target_policy,nS,nA)
    #batch_size = 50;
    obj = [average_case_distribution(nS, nA, behaviour_policy, start, lr,batch_size) for i in range(nPOL)]
    for index,pol in enumerate(one_hot_target_policy):
        obj[index].set_target_policy(pol);
    data = simulate_episode(T*batch_size,start,behaviour_policy,P,batch_size,run);
    #print(data)
    #input();
    #theta = np.ones(nPOL);
    result = []
    sample=np.zeros(T)
    n=np.ones(nPOL)
    S = np.ones(nPOL);
    F = np.ones(nPOL);
    rew = np.zeros(nPOL)
    #data_used=[]
    for t in tqdm(range(1,T+1)):
        sampled_policy = np.argmin([np.random.beta(S[j],F[j]) for j in range(nPOL)])
        #sampled_policy = np.argmax(np.random.multinomial(1,P_i))
        obj[sampled_policy].set_batch(data[t-1])
        sample[t-1] = sampled_policy
        obj[sampled_policy].update_state_distribution_ratio()
        rho_vect = get_rho(obj,nPOL,nS,nA,R,behaviour_policy_state_distribution,target_policy,sampled_policy)
        rew[sampled_policy] = (1-alpha)*rew[sampled_policy] + alpha*rho_vect[sampled_policy];
        S[sampled_policy]+=rew[sampled_policy];
        F[sampled_policy]=F[sampled_policy]+1-rew[sampled_policy]
        result.append(rho_vect[sampled_policy]);
    with open('sampled_policy_'+str(run)+'_thompson_sampling','wb') as f:
        pickle.dump(sample,f);
    f.close();
    with open('estimated_value_function_'+str(run),'wb') as f:
        pickle.dump(result,f);
    f.close();
    return S,F

In [25]:
if __name__=='__main__':
    T,runs,lr,batch_size,start,nS,nA,rep_cost,alpha,gamma,beta = get_hyperparameters().ret_hyperparameters();
    nPOL = nS
    T_update = int(T/batch_size);
    mr_obj = Machine_Replacement(rep_cost,nS,nA);
    P,R = mr_obj.gen_probability(),mr_obj.gen_expected_reward()
    behaviour_policy = np.ones((nS,nA))*0.5
    behaviour_policy_state_distribution = beh_pol_sd(P,behaviour_policy,nS,nA).state_distribution_simulated(0);
    #policy_sampled = np.zeros((T_update,runs))
    target_policy = np.ones((nPOL,nS),dtype = np.int8)
    #data_dict={0:[],1:[],2:[],3:[],4:[]};
    #data_used={0:[],1:[],2:[],3:[],4:[]};
    for i in range(nPOL-1,0,-1):
        target_policy[nPOL-i-1][0:i] = 0;
    
    #mp.set_start_method('spawn')
    S0,F0=processing(P,R,nPOL,nS,nA,T_update,behaviour_policy,behaviour_policy_state_distribution,target_policy,0,lr,beta,batch_size,alpha);
    S1,F1=processing(P,R,nPOL,nS,nA,T_update,behaviour_policy,behaviour_policy_state_distribution,target_policy,1,lr,beta,batch_size,alpha);
    S2,F2=processing(P,R,nPOL,nS,nA,T_update,behaviour_policy,behaviour_policy_state_distribution,target_policy,2,lr,beta,batch_size,alpha);
    S3,F3=processing(P,R,nPOL,nS,nA,T_update,behaviour_policy,behaviour_policy_state_distribution,target_policy,3,lr,beta,batch_size,alpha);
    S4,F4=processing(P,R,nPOL,nS,nA,T_update,behaviour_policy,behaviour_policy_state_distribution,target_policy,4,lr,beta,batch_size,alpha);
    
    '''p1 = mp.Process(target=processing,args=(P,R,nPOL,nS,nA,T_update,behaviour_policy,behaviour_policy_state_distribution,target_policy,0,lr,beta))
    p2 = mp.Process(target=processing,args=(P,R,nPOL,nS,nA,T_update,behaviour_policy,behaviour_policy_state_distribution,target_policy,1,lr,beta))
    p3 = mp.Process(target=processing,args=(P,R,nPOL,nS,nA,T_update,behaviour_policy,behaviour_policy_state_distribution,target_policy,2,lr,beta))
    p4 = mp.Process(target=processing,args=(P,R,nPOL,nS,nA,T_update,behaviour_policy,behaviour_policy_state_distribution,target_policy,3,lr,beta))
    p5 = mp.Process(target=processing,args=(P,R,nPOL,nS,nA,T_update,behaviour_policy,behaviour_policy_state_distribution,target_policy,4,lr,beta))
    p1.start()
    p2.start()
    p3.start()
    p4.start()
    p5.start()
    p1.join();
    p2.join();
    p3.join();
    p4.join();
    p5.join();'''

100%|█████████████████████████████████████| 5000/5000 [2:02:21<00:00,  1.47s/it]


In [14]:
policy_sampled={0:[],1:[],2:[]}
runs = 3
for run in range(runs):
    sample=[];
    with open('sampled_policy_'+str(run)+'_thompson_sampling','rb') as f:
        sample = pickle.load(f);
    f.close();
    policy_sampled[run]=np.array(sample);
pd.DataFrame(policy_sampled).to_excel('Policy_sampled_'+str(nS)+'_states_Thompson_Sampling.xlsx');

In [13]:
S0/(S0+F0)

array([0.38215539, 0.37549289, 0.30621877, 0.49972692, 0.36201326,
       0.37442329, 0.32021619, 0.52234472, 0.60228721, 0.69233774])

In [14]:
S1/(S1+F1)

array([0.46178258, 0.38981396, 0.34727536, 0.43018664, 0.38157111,
       0.44778206, 0.4151543 , 0.49378128, 0.62576574, 0.69233774])

In [17]:
S2/(S2+F2)

array([0.43074124, 0.44319879, 0.4425616 , 0.46862673, 0.48527113,
       0.4844014 , 0.45515656, 0.47444201, 0.57679075, 0.74615385])

In [18]:
S3/(S3+F3)

array([0.45881842, 0.36848613, 0.46053002, 0.64875558])

In [19]:
S4/(S4+F4)

array([0.47871307, 0.43200961, 0.49286575, 0.65442737])

In [34]:
#10 states
print(S0/(S0+F0))
print(S1/(S1+F1))
print(S2/(S2+F2))
print(S3/(S3+F3))
print(S4/(S4+F4))

[0.40750662 0.39188426 0.41261151 0.40046216 0.44873811 0.45185478
 0.46744044 0.48952903 0.50984941 0.62124366]
[0.41567177 0.40202542 0.44816743 0.43016518 0.44758695 0.42555106
 0.45689012 0.48121394 0.52613041 0.6289497 ]
[0.46381073 0.46351354 0.40193494 0.45181136 0.42704374 0.47064447
 0.47909463 0.45805703 0.54370631 0.64265049]
[0.40417055 0.40402534 0.46616638 0.42859453 0.38841131 0.43849415
 0.45943219 0.4909683  0.50103243 0.6289497 ]
[0.42256612 0.39586538 0.41446995 0.44902604 0.44169738 0.42363119
 0.48947279 0.47069462 0.51726009 0.64265049]
